<a href="https://colab.research.google.com/github/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/FlashRerankPractical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Model Options:**
- **Nano**: ~4MB, blazing fast model with competitive performance (ranking precision).
- **Small**: ~34MB, slightly slower with the best performance (ranking precision).
- **Medium**: ~110MB, slower model with the best zero-shot performance (ranking precision).
- **Large**: ~150MB, slower model with competitive performance (ranking precision) for 100+ languages.

 **Flash Rank**: Ultra-lite & Super-fast Python library for search & retrieval re-ranking.

- **Ultra-lite**: No heavy dependencies. Runs on CPU with a tiny ~4MB reranking model.
- **Super-fast**: Speed depends on the number of tokens in passages and query, plus model depth.
- **Cost-efficient**: Ideal for serverless deployments with low memory and time requirements.
- **Based on State-of-the-Art Cross-encoders**: Includes models like ms-marco-TinyBERT-L-2-v2 (default), ms-marco-MiniLM-L-12-v2, rank-T5-flan, and ms-marco-MultiBERT-L-12.
- **Sleek Models for Efficiency**: Designed for minimal overhead in user-facing scenarios.

_Flash Rank is tailored for scenarios requiring efficient and effective reranking, balancing performance with resource usage._

In [ ]:
!pip install flashrank

In [ ]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i+1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [ ]:
query = "How to speedup LLMs?"

In [ ]:
passages = [
   {
      "id":1,
      "text":"Introduce *lookahead decoding*: - a parallel decoding algo to accelerate LLM inference - w/o the need for a draft model or a data store - linearly decreases # decoding steps relative to log(FLOPs) used per decoding step.",
      "meta": {"additional": "info1"}
   },
   {
      "id":2,
      "text":"LLM inference efficiency will be one of the most crucial topics for both industry and academia, simply because the more efficient you are, the more $$$ you will save. vllm project is a must-read for this direction, and now they have just released the paper",
      "meta": {"additional": "info2"}
   },
   {
      "id":3,
      "text":"There are many ways to increase LLM inference throughput (tokens/second) and decrease memory footprint, sometimes at the same time. Here are a few methods I’ve found effective when working with Llama 2. These methods are all well-integrated with Hugging Face. This list is far from exhaustive; some of these techniques can be used in combination with each other and there are plenty of others to try. - Bettertransformer (Optimum Library): Simply call `model.to_bettertransformer()` on your Hugging Face model for a modest improvement in tokens per second. - Fp4 Mixed-Precision (Bitsandbytes): Requires minimal configuration and dramatically reduces the model's memory footprint. - AutoGPTQ: Time-consuming but leads to a much smaller model and faster inference. The quantization is a one-time cost that pays off in the long run.",
      "meta": {"additional": "info3"}

   },
   {
      "id":4,
      "text":"Ever want to make your LLM inference go brrrrr but got stuck at implementing speculative decoding and finding the suitable draft model? No more pain! Thrilled to unveil Medusa, a simple framework that removes the annoying draft model while getting 2x speedup.",
      "meta": {"additional": "info4"}
   },
   {
      "id":5,
      "text":"vLLM is a fast and easy-to-use library for LLM inference and serving. vLLM is fast with: State-of-the-art serving throughput Efficient management of attention key and value memory with PagedAttention Continuous batching of incoming requests Optimized CUDA kernels",
      "meta": {"additional": "info5"}
   }
]


In [ ]:
from flashrank.Ranker import Ranker, RerankRequest

In [ ]:
def get_result(query,passages,choice):
  if choice == "Nano":
    ranker = Ranker()
  elif choice == "Small":
    ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2", cache_dir="/opt")
  elif choice == "Medium":
    ranker = Ranker(model_name="rank-T5-flan", cache_dir="/opt")
  elif choice == "Large":
    ranker = Ranker(model_name="ms-marco-MultiBERT-L-12", cache_dir="/opt")
  rerankrequest = RerankRequest(query=query, passages=passages)
  results = ranker.rerank(rerankrequest)
  print(results)

  return results

In [ ]:
%%time
print("sunny")

In [ ]:
%%time
get_result(query,passages,"Nano")

In [ ]:
%%time
get_result(query,passages,"Small")

In [ ]:
%%time
get_result(query,passages,"Medium")

In [ ]:
!pip install langchain_community

In [ ]:
!pip install langchain_openai

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
documents = TextLoader("/content/state_of_the_union.txt").load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [ ]:
texts = text_splitter.split_documents(documents)

In [ ]:
for id, text in enumerate(texts):
    text.metadata["id"] = id

In [ ]:
texts

In [ ]:
embedding = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
!pip install faiss-cpu

In [ ]:
retriever = FAISS.from_documents(texts, embedding).as_retriever(search_kwargs={"k": 10})

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"

In [ ]:
docs = retriever.invoke(query)

In [ ]:
pretty_print_docs(docs)

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
compressor = FlashrankRerank()

In [ ]:
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

In [ ]:
compressed_docs = compression_retriever.invoke("What did the president say about Ketanji Jackson Brown")

In [ ]:
len(compressed_docs)


In [ ]:
compressed_docs

In [ ]:
print([doc.metadata["id"] for doc in compressed_docs])

In [ ]:
pretty_print_docs(compressed_docs)

In [ ]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, retriever=compression_retriever)

In [ ]:
chain.invoke(query)